<a href="https://colab.research.google.com/github/ylistokin/UNGA/blob/main/Annual_Country_Resolutions_MV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This file counts how many resolutions criticize a country in each year from 1981-2024.
 1.  It first appends the Voelsen GPT evaluated data from /content/gdrive/MyDrive/UNGA/full_text_data2.csv. and the Mesquita GPT evaluated data from /content/gdrive/MyDrive/UNGA/MesquitaGPT.csv
 2. Because of possible double countring in Voelsen and Mesquita due to treated different sections of a single resolution as multiple resolutions, the file creates two datasets, one with the observations as counted by Voelsen and Mesquita and the other with only one observations for each UNGA number, thereby eliminating multiple sections.
 3. After appending the two datasets, I process the country names so that they are consistent and disentangle observations where more than one country is criticized.
 4. Finally, the data is counted by year and country to give the counts.
 5. Recall that most countries are not criticized in a particular year. T
  After this To do this correctly, I need to account for resolutions criticizing more than one country. I save the data in /content/gdrive/MyDrive/UNGA/GPT_resolution_counts.csv. This data does not include counts for countries that are not criticized in a particular year.

In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
from tqdm import tqdm


from google.colab import drive
drive.mount("/content/gdrive")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Reading in Voelsen Resolution data that has already been labeled by GPT
df1 = pd.read_csv("/content/gdrive/MyDrive/UNGA/full_text_data2.csv")

#Reading in Mesquita Resolution data that has already been labeled by GPT
df2 = pd.read_csv("/content/gdrive/MyDrive/UNGA/MesquitaGPT.csv")


<ipython-input-49-876fc3af06d1>:2: DtypeWarning: Columns (48,302,304,305,306,307) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("/content/gdrive/MyDrive/UNGA/full_text_data2.csv")


In [ ]:
# Vertically append the DataFrames
df = pd.concat([df1, df2], axis=0, ignore_index=True)



In [ ]:
print(df.shape)
print(df1.shape)
print(df2.shape)
#print(df['year'].value_counts())

(14095, 328)
(9809, 309)
(4286, 20)


In [ ]:
df.head()
# Check for missing values
missing_values = df['year'].isnull().sum()
print("Missing values in each column:")
print(missing_values)



Missing values in each column:
4286


In [ ]:
#Replacing None with missing, which is more relevant for my purposes here
df['country'] = df['country'].fillna('None')
df['Country Concerned'] = df['Country Concerned'].fillna('None')

#Getting rid of extraneous spaces before and after name
df['country'] = df['country'].str.strip()

# Drop rows where 'Country' contains "Understood"
df = df[~df['country'].str.contains("Understood", case=False, na=False)]

In [ ]:
#Fixing up country names when GPT identifies more than one country being criticized, with the country names separated by commas

#Duplicating observations when there are multiple countries condemned so that each country condemned is counted
df['comma_count'] = df['country'].fillna('').str.count(',')+1
print(df['comma_count'].value_counts())

#subdividing when there is a comma
df['country1'] = df['country'].str.split(", ").str[0]
df['country2'] = df['country'].str.split(", ").str[1]
df['country3'] = df['country'].str.split(", ").str[2]
df['country4'] = df['country'].str.split(", ").str[3]
df['country5'] = df['country'].str.split(", ").str[4]

df['country1'] = df['country1'].str.strip()
# Check equality between the two columns
equal_count = (df['country'] == df['country1']).sum()
unequal_count = (df['country'] != df['country1']).sum()

# Display the results
print(f"Equal count: {equal_count}")
print(f"Unequal count: {unequal_count}")

# Filter rows where 'column_a' is not equal to 'column_b'
unequal_rows = df[df['country'] != df['country1']]
equal_rows = df[df['country'] == df['country1']]




comma_count
1    14000
2       30
3       11
5        3
4        2
Name: count, dtype: int64
Equal count: 14000
Unequal count: 46


In [ ]:
#Working with the unequal_count dataframe, which has all the observations with multiple countries being condemned
print(unequal_rows['comma_count'].value_counts())
print(unequal_rows.shape)
#Duplicating observations by the number of commas
unequal_rows = unequal_rows.loc[unequal_rows.index.repeat(unequal_rows['comma_count'])].reset_index(drop=True)
unequal_rows['order'] = unequal_rows.groupby('unres').cumcount() + 1
print(unequal_rows.shape)




comma_count
2    30
3    11
5     3
4     2
Name: count, dtype: int64
(46, 334)
(116, 335)


In [ ]:
#Creating a new variable, country, that equals country1 country2, country3 or country4, based on the value of order
# Define conditions and corresponding values for n
conditions = [
    (unequal_rows['order'] == 1),
    (unequal_rows['order'] == 2),
    (unequal_rows['order'] == 3),
    (unequal_rows['order'] == 4),
    (unequal_rows['order'] == 5)
]
values = [unequal_rows['country1'], unequal_rows['country2'], unequal_rows['country3'], unequal_rows['country4'], unequal_rows['country5'] ]

# Create the new variable n
unequal_rows['country'] = np.select(conditions, values, default=np.nan)


In [ ]:
unequal_rows[['country', 'country1', 'country2', 'country3', 'order']].head(20)

,country,country1,country2,country3,order
0,Comoros,Comoros,Sao Tome and Principe,Somalia,1.0
1,Sao Tome and Principe,Comoros,Sao Tome and Principe,Somalia,2.0
2,Somalia,Comoros,Sao Tome and Principe,Somalia,3.0
3,Comoros,Comoros,Sao Tome and Principe,Somalia,1.0
4,Sao Tome and Principe,Comoros,Sao Tome and Principe,Somalia,2.0
5,Somalia,Comoros,Sao Tome and Principe,Somalia,3.0
6,Comoros,Comoros,Sao Tome and Principe,Somalia,1.0
7,Sao Tome and Principe,Comoros,Sao Tome and Principe,Somalia,2.0
8,Somalia,Comoros,Sao Tome and Principe,Somalia,3.0
9,Comoros,Comoros,Sao Tome and Principe,Somalia,1.0


In [ ]:
#Combinging the data that has just been manipulated from the data frame unequal rows with the main data from equal rows.
df = pd.concat([equal_rows, unequal_rows], ignore_index=True)

In [ ]:
#Fixing country names

df['country'] = df['country'].replace("Russian Federation", "Russia")
df['country'] = df['country'].replace("Islamic Republic of Iran", "Iran")
df['country'] = df['country'].replace("Republic of Korea", "South Korea")
df['country'] = df['country'].replace("Democratic People’s Republic of Korea", "Democratic People's Republic of Korea")

df['country'] = df['country'].replace("Republic of Moldova", "Moldova")
df['country'] = df['country'].replace("United States of America", "United States")
df['country'] = df['country'].replace("Syrian Arab Republic", "Syria")
df['country'] = df['country'].replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom")
df['country'] = df['country'].replace("Taliban", "Afghanistan")
df['country'] = df['country'].replace("Democratic Peoples Republic of Korea", "North Korea")
df['country'] = df['country'].replace("Federal Republic of Yugoslavia", "Serbia")
df['country'] = df['country'].replace("Serbia and Montenegro", "Serbia")
df['country'] = df['country'].replace("Federal Republic of Yugoslavia (Serbia and Montenegro)", "Serbia")
df['country'] = df['country'].replace("Côte d'Ivoire", "Cote d'Ivoire")
df['country'] = df['country'].replace("Côte d’Ivoire", "Cote d'Ivoire")
df['country'] = df['country'].replace("South Africa and Namibia", "South Africa")
df['country'] = df['country'].replace("Libya Arab Jamahiriya", "Libya")

In [ ]:
#I don't want "None" to be a country
df['country'] = df['country'].replace("None", np.nan)



In [ ]:
#Fixing the year variable for the data originally from Mesquita
# Replace missing 'year' values with 1945 + 'session_reg'
df['year'] = df['year'].fillna(1945 + df['session_reg'])



In [ ]:
# Count the number of resolutions criticizing each country in each year, adding up the observations where there are multiple countries
resolution_counts = df.groupby(['country', 'year']).size().reset_index(name='count')
resolution_counts.head()




,country,year,count
0,Afghanistan,1985.0,1
1,Afghanistan,1986.0,1
2,Afghanistan,1987.0,1
3,Afghanistan,1988.0,1
4,Afghanistan,1989.0,1


In [ ]:
file_path = '/content/gdrive/MyDrive/UNGA/GPT_resolution_counts.csv'

resolution_counts.to_csv(file_path, index=False)

#Everything below this deals with having a single observation for each unique resolution, which is generally a waste of time.

In [ ]:
# Create the new variable `res_symbol`
df1['resolution_num_short'] = df1['unres'].apply(
    lambda x: x[:-1] if x[-1].isalpha() else x
)
not_equal_df1 = df1[df1['resolution_num_short'] != df1['unres']][['unres', 'resolution_num_short']]
not_equal_filtered = not_equal_df1.drop_duplicates(subset='resolution_num_short', keep='first')
equal_df1=df1[df1['resolution_num_short'] == df1['unres']]
df1_unique=pd.concat([equal_df1,not_equal_filtered])
print(df1.shape)
print(df1_unique.shape)
print(df1['country'].value_counts().head(5))
print(df1_unique['country'].value_counts().head(5))



(9809, 310)
(9096, 310)
country
Israel                                                  536
United States of America                                 87
United Kingdom of Great Britain and Northern Ireland     59
Democratic People’s Republic of Korea                    49
Syria                                                    34
Name: count, dtype: int64
country
Israel                                                  504
United States of America                                 83
United Kingdom of Great Britain and Northern Ireland     57
Democratic People’s Republic of Korea                    49
Syria                                                    32
Name: count, dtype: int64


In [ ]:
# Keep only one observation for each unique 'res_id2_unlet' value to avoid potential double counting of single resolutions with multiple sections
df2_unique = df2.drop_duplicates(subset='res_id2_unlet', keep='first')

print(df2.shape)
print(df2['country'].value_counts().head(5))

print(df2_unique.shape)
print(df2_unique['country'].value_counts().head(5))

(4286, 20)
country
South Africa                318
Israel                      298
United States of America     55
France                       27
United Kingdom               23
Name: count, dtype: int64
(3182, 20)
country
South Africa                184
Israel                       86
United States of America     52
France                       27
United Kingdom               23
Name: count, dtype: int64


In [ ]:
# Keep only one observation for each unique 'res_id2_unlet' value to avoid potential double counting of single resolutions with multiple sections
df2_unique = df2.drop_duplicates(subset='res_id2_unlet', keep='first')
print(df2_unique.shape)
print(df2_unique['country'].value_counts())

(3182, 20)
country
South Africa                                              184
Israel                                                     86
United States of America                                   52
France                                                     27
United Kingdom                                             23
United Kingdom of Great Britain and Northern Ireland       21
Sudan                                                      17
Morocco                                                    13
El Salvador                                                12
Afghanistan                                                11
Iraq                                                       10
Chile                                                      10
Ethiopia                                                    9
Haiti                                                       9
Somalia                                                     9
Nicaragua                                          

In [ ]:
# Vertically append the DataFrames for the dataframes with only unique resolution numbers
df_unique = pd.concat([df1_unique, df2_unique], axis=0, ignore_index=True)

In [ ]:
#Doing the same thing for df_unique, which is more relevant for my purposes here
df_unique['country'] = df_unique['country'].fillna('None')
df['Country Concerned'] = df['Country Concerned'].fillna('None')

#Getting rid of extraneous spaces before and after name
df_unique['country'] = df_unique['country'].str.strip()
print(df_unique.shape)

#Getting rid of all observations with commas
df_unique = df_unique[~df_unique['country'].str.contains(",")]
print(df_unique.shape)

(12278, 329)
(12245, 329)


<ipython-input-95-6eaa4341e0f8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country Concerned'] = df['Country Concerned'].fillna('None')


In [ ]:
unequal_rows['order'].value_counts()

,count
order,
1.0,32
2.0,32
3.0,16
4.0,5
5.0,3


In [ ]:
df.shape

(14116, 336)

In [ ]:
#Doing the same for df_unique
df_unique['country'] = df_unique['country'].replace("Russian Federation", "Russia")
df_unique['country'] = df_unique['country'].replace("Islamic Republic of Iran", "Iran")
df_unique['country'] = df_unique['country'].replace("Republic of Korea", "South Korea")
df_unique['country'] = df_unique['country'].replace("Democratic People's Republic of Korea", "North Korea")
df_unique['country'] = df_unique['country'].replace("Republic of Moldova", "Moldova")
df_unique['country'] = df_unique['country'].replace("United States of America", "United States")
df_unique['country'] = df_unique['country'].replace("Syrian Arab Republic", "Syria")
df_unique['country'] = df_unique['country'].replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom")
df_unique['country'] = df_unique['country'].replace("Taliban", "Afghanistan")
df_unique['country'] = df_unique['country'].replace("Democratic People’s Republic of Korea", "North Korea")
df_unique['country'] = df_unique['country'].replace("Federal Republic of Yugoslavia", "Serbia and Montenegro")
df_unique['country'] = df_unique['country'].replace("Federal Republic of Yugoslavia (Serbia and Montenegro)", "Serbia and Montenegro")


In [ ]:
df_unique['country'] = df_unique['country'].replace("None", np.nan)
df_unique['year'] = df_unique['year'].fillna(1945 + df['session_reg'])
resolution_counts_unique = df_unique.groupby(['country', 'year']).size().reset_index(name='count')

In [ ]:
#Saving the files to Google Drive

file_path = '/content/gdrive/MyDrive/UNGA/GPT_resolution_counts.csv'
file_path2= '/content/gdrive/MyDrive/UNGA/GPT_unique_resolution_counts.csv'

resolution_counts.to_csv(file_path, index=False)
resolution_counts_unique.to_csv(file_path2, index=False)